<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/128_Docker_Container_Creation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🚀 Docker Project Checklist (1-Page)

### 📂 Project Setup

```bash
mkdir myproject && cd myproject   # create project folder
nano Dockerfile                   # create Dockerfile
nano .dockerignore                 # create .dockerignore
nano requirements.txt              # runtime deps
nano requirements-dev.txt          # dev tooling
nano main.py                       # entry/test script
nano claude_chat.py                # (your Claude helper functions)
nano .env                          # API keys (local only, not copied into image)
```

### 📄 Typical Files

* **Dockerfile** → recipe for environment
* **.dockerignore** → exclude junk/secrets
* **requirements.txt** → runtime Python deps
* **requirements-dev.txt** → dev tooling
* **main.py** → minimal entrypoint/test script
* **claude\_chat.py** → reusable Claude helper functions
* **.env** → API keys (local only, safe from Git/Docker builds)

---

### 🛠️ Build & Run

```bash
# Build runtime image
docker build -t myproject:latest .

# Build dev image (with Jupyter, pytest, etc.)
docker build -t myproject:dev --build-arg INSTALL_DEV=true .

# Run runtime app with secrets
docker run --rm --env-file .env myproject:latest

# Run with interactive shell
docker run -it myproject:latest bash

# Run Jupyter in dev mode
docker run --rm -it -p 8888:8888 myproject:dev \
    jupyter lab --ip=0.0.0.0 --allow-root
```

### 🧹 Cleanup (optional)

```bash
docker builder prune      # clear build cache
docker system prune -a    # remove all unused containers/images
```






## 🧠 Why Containers Are Great for AI Agents

1. **Isolation**

   * Keeps your dependencies separate from other projects.
   * You can have one container using Claude, another using OpenAI, another for experiments.

2. **Reproducibility**

   * If you get an agent working today, you can still rebuild the same environment 6 months later.

3. **Safety with Secrets**

   * Your `.env` keys (Claude, OpenAI, etc.) never get baked into the image.
   * You inject them at runtime, so they’re safe and private.

4. **Portability**

   * The same container works on your Mac, a cloud server, or even someone else’s laptop — no “but it worked on my machine” issues.



# 🚀 Docker Project Template for Data/ML Projects

## 📂 Folder Structure

Every new project starts like this:

```
myproject/
├── .dockerignore
├── Dockerfile
├── requirements.txt
├── requirements-dev.txt
└── main.py
```

(Optional: add `src/` and `notebooks/` as your project grows.)

```
src/
  test_claude.py
  data_pipeline.py
  agent_loop.py
```

In [ ]:
myproject/
├── .dockerignore          # excludes junk/secrets from images
├── Dockerfile             # recipe for building container
├── requirements.txt       # runtime dependencies (Claude, rich, etc.)
├── requirements-dev.txt   # developer dependencies (Jupyter, pytest, etc.)
├── .env                   # API keys (never committed, ignored by Git)
│
├── main.py                # minimal entrypoint (system check + hello Claude)
│
├── claude_chat.py         # Claude chat helper functions (multi-turn chat, pretty printing)
│
└── src/                   # your experiments & agent scripts
    ├── chat_loop.py       # interactive REPL with Claude
    ├── data_pipeline.py   # future data/ML experiments
    └── agent_loop.py      # agent orchestration logic


# Build Container Requirement Docs


## 📄 1. `.dockerignore`

Keeps junk, secrets, and large data out of your images.


In [ ]:
nano .dockerignore # then paste the following to create the doc

In [ ]:
# Python cache
__pycache__/
*.pyc
*.pyo
*.pyd

# Virtual environments
venv/
env/
.venv/

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# OS files
.DS_Store
Thumbs.db

# Git and version control
.git
.gitignore
.gitattributes

# Logs & debug
*.log
*.out
*.err

# Data & models (better to mount at runtime!)
data/
datasets/
*.csv
*.parquet
*.h5
*.pkl

# Large results
outputs/
results/
checkpoints/

# Secrets
.env
*.secret
*.key


## 📄 2. `requirements.txt`

**Runtime essentials** (only what your app needs to run):

* **anthropic** → Claude API client
* **rich** → pretty terminal output (markdown, code blocks)
* **numpy / pandas / scikit-learn** → ML/data science stack (optional, but great if you’re doing experiments)




In [ ]:
nano requirements.txt

In [ ]:
anthropic>=0.37.0
rich==13.7.1
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.5.1

## 📄 3. `requirements-dev.txt`

**Developer extras** (useful for notebooks, testing, formatting):


In [ ]:
nano requirements-dev.txt

In [ ]:
jupyterlab==4.3.0
ipykernel==6.29.5
pytest==8.3.3
flake8==7.1.1
black==24.8.0

## 📄 4. `Dockerfile`

The full environment recipe:

---

## 🔎 What each part does

1. **Base image**

   ```dockerfile
   FROM python:3.11-slim
   ```

   * Starts with a lightweight Python 3.11 image (no extra OS fluff).

2. **Environment settings**

   ```dockerfile
   ENV PYTHONDONTWRITEBYTECODE=1
   ENV PYTHONUNBUFFERED=1
   ```

   * Prevents `.pyc` files clutter.
   * Ensures Python prints show up immediately in logs (important inside containers).

3. **System dependencies**

   ```dockerfile
   RUN apt-get update && apt-get install -y --no-install-recommends \
       build-essential \
       curl \
       git \
       wget \
       ca-certificates \
       && rm -rf /var/lib/apt/lists/*
   ```

   * Installs minimal but common tools (compilers, git, wget, certs).
   * Cleans up after itself to keep image size small.

4. **Working directory**

   ```dockerfile
   WORKDIR /app
   ```

   * All following operations happen inside `/app`.
   * When the container runs, you’ll start here.

5. **Dependencies (two-step install)**

   ```dockerfile
   COPY requirements.txt requirements-dev.txt ./
   RUN pip install --no-cache-dir -r requirements.txt
   ```

   * Copies only requirements files first → better caching (you don’t re-download everything unless requirements change).
   * Installs runtime dependencies.

6. **Optional dev dependencies**

   ```dockerfile
   ARG INSTALL_DEV=false
   RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi
   ```

   * Lets you control at build time if dev deps are installed.
   * Normal build → runtime only (`myproject:latest`).
   * Dev build → adds Jupyter, pytest, etc. (`myproject:dev`).

7. **Copy your code**

   ```dockerfile
   COPY . .
   ```

   * Now brings in everything else (except stuff ignored by `.dockerignore`).

8. **Default run command**

   ```dockerfile
   CMD ["python", "main.py"]
   ```

   * When you do `docker run myproject:latest`, this is what executes.
   * You can override it (e.g. run bash or Jupyter) if needed.



In [ ]:
nano Dockerfile

In [ ]:
FROM python:3.11-slim

# Prevents pyc files, forces flush of stdout/stderr
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# System deps
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Workdir
WORKDIR /app

# Copy dependency files first (better cache)
COPY requirements.txt requirements-dev.txt ./

# Install runtime deps
RUN pip install --no-cache-dir -r requirements.txt

# Optional dev deps
ARG INSTALL_DEV=false
RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi

# Copy code
COPY . .

# Default run
CMD ["python", "main.py"]



## 📄 5. `main.py`

This should stay **minimal**. Its job is to:

* Confirm your container runs correctly (print Python version, OS, args).
* Do a quick “hello world” test with Claude, using helper functions from `claude_chat.py`.
* Act as the **entrypoint** your Dockerfile runs by default.



In [ ]:
nano main.py

In [ ]:
import sys, platform, os
from claude_chat import chat_with_claude, reset_conversation

def main():
    name = os.getenv("NAME", "world")
    print(f"Hello, {name} 👋")
    print("Python:", platform.python_version())
    print("OS:", platform.platform())
    print("Args:", sys.argv[1:])

    reset_conversation()
    chat_with_claude("Hello Claude, can you confirm you’re working?")

if __name__ == "__main__":
    main()




## 📄 6. `claude_chat.py`

This file holds all the **Claude helper functions** so `main.py` can stay clean.

It provides:

* **`chat_with_claude()`** → send prompts and print responses nicely.
* **Conversation memory** (so multi-turn chats work).
* **`reset_conversation()`** and **`last_reply()`** helpers.
* Safe API key handling (loaded at runtime from `.env`).

Keeping this logic separate makes your project easier to extend and keeps secrets out of your code.



In [ ]:
nano claude_chat.py

In [2]:
import os, textwrap
from anthropic import Anthropic
from anthropic._exceptions import APIError
from rich.console import Console
from rich.markdown import Markdown

# API setup
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY environment variable.")

client = Anthropic(api_key=anthropic_key)
MODEL_NAME = "claude-3-5-sonnet-20240620"

# Conversation history
conversation = []

# Pretty printing setup
console = Console()

def smart_print_markdown(output: str, width: int = 100):
    """Wrap plain text, preserve fenced code blocks."""
    in_code = False
    para_buf = []

    def flush_paragraph():
        if para_buf:
            text = " ".join(para_buf)
            print(textwrap.fill(text, width=width, replace_whitespace=False))
            print()
            para_buf.clear()

    for line in output.splitlines():
        fence = line.strip().startswith("```")
        if fence:
            flush_paragraph()
            print(line)
            in_code = not in_code
            continue

        if in_code:
            print(line)
        else:
            if line.strip() == "":
                flush_paragraph()
            else:
                para_buf.append(line)

    flush_paragraph()

def chat_with_claude(
    prompt: str,
    system: str = "You are a helpful coding assistant.",
    render: str = "markdown",      # 'markdown' | 'wrapped' | 'none'
    return_text: bool = False,
    wrap_width: int = 100,
) -> str | None:
    """Send a prompt with conversation memory."""
    if not anthropic_key:
        raise RuntimeError("Missing ANTHROPIC_API_KEY.")

    conversation.append({"role": "user", "content": prompt})

    try:
        msg = client.messages.create(
            model=MODEL_NAME,
            max_tokens=1000,
            temperature=0.2,
            system=system,
            messages=conversation,
        )
        parts = [b.text for b in msg.content if getattr(b, "type", None) == "text"]
        output = "\n\n".join(parts).strip() or "(No text)"

        if render == "markdown":
            console.print(Markdown(output))
        elif render == "wrapped":
            smart_print_markdown(output, width=wrap_width)

        conversation.append({"role": "assistant", "content": output})
        return output if return_text else None

    except APIError as e:
        print("Anthropic API error:", e)
        raise

def reset_conversation():
    """Clear conversation history."""
    conversation.clear()

def last_reply() -> str | None:
    """Get last assistant message."""
    for m in reversed(conversation):
        if m["role"] == "assistant":
            return m["content"]
    return None





## 🔐 7. `.env`

This file stores your **secret environment variables** (like API keys).

* It should **never** be copied into your Docker image (your `.dockerignore` already protects it).
* Instead, you load it at **runtime** with the `--env-file .env` flag when running a container.
* This way, your keys stay safe and out of version control.

### Create it

```bash
nano .env
```

Paste your key (replace with your real value):

```txt
ANTHROPIC_API_KEY=your_api_key_here
```

### Usage

When running your container:

```bash
docker run --rm --env-file .env myproject:latest
```

Now `claude_chat.py` can access the key with:

```python
os.getenv("ANTHROPIC_API_KEY")
```




In [ ]:
nano.env

In [ ]:
ANTHROPIC_API_KEY=your_api_key_here

In [ ]:
myproject/
├── .dockerignore
├── Dockerfile
├── requirements.txt
├── requirements-dev.txt
├── main.py
├── claude_chat.py   👈 new
└── .env             👈 local only (not copied into image)




# 🛠️ Step 8: Building Docker Images

When you run:

```bash
docker build -t myproject:latest .
```

you’re telling Docker:

1. **Read my `Dockerfile`** → it’s the recipe.

2. **Use the current directory (`.`)** as the **build context** → Docker can “see” the files here (but it will ignore anything in `.dockerignore`).

3. **Follow the instructions step by step**:

   * Start from `python:3.11-slim`.
   * Install system dependencies (curl, git, etc.).
   * Copy in `requirements.txt` and `requirements-dev.txt`.
   * Install runtime Python packages from `requirements.txt`.
   * Optionally install dev packages if `INSTALL_DEV=true`.
   * Copy in all your project code (`main.py`, `claude_chat.py`).
   * Set the default command → `python main.py`.

4. **Save the result as an image** → tagged `myproject:latest`.

   * This is a **frozen snapshot of your environment** (OS + Python + libraries + your code).

---

## 🛠️ Dev Image (optional extras)

When you run:

```bash
docker build -t myproject:dev --build-arg INSTALL_DEV=true .
```

* Same steps as above, **plus** it installs `requirements-dev.txt`.
* That means Jupyter, pytest, linters, etc. get baked into the image.
* Tag is `myproject:dev` so you can tell them apart.

---

# 🚀 Why this matters

* An **image** is like a *template* (frozen recipe).
* A **container** is a *running instance* of an image.





In [ ]:
# Build the runtime image
docker build -t myproject:latest .

# Build the dev image (includes Jupyter, pytest, etc.)
docker build -t myproject:dev --build-arg INSTALL_DEV=true .




### 🐳 Why two build steps?

You’re really building **two different flavors** of the same project:

---

### 1. **Runtime Image**

```bash
docker build -t myproject:latest .
```

* **Purpose**: Production / deployment.
* **Contents**:

  * Python base image.
  * System deps (git, curl, etc.).
  * `requirements.txt` only → just what’s needed for the app to run (`anthropic`, `rich`, `numpy`, etc.).
  * Your code (`main.py`, `claude_chat.py`).
* **Result**: Small, lean image → quicker to build, lighter to run.

This is what you’d actually deploy to a server or share with others.

---

### 2. **Dev Image**

```bash
docker build -t myproject:dev --build-arg INSTALL_DEV=true .
```

* **Purpose**: Local development / experimentation.
* **Contents**:

  * Everything in runtime.
  * Plus installs `requirements-dev.txt` → JupyterLab, pytest, flake8, black, etc.
* **Result**: Bigger image, but with all the tools you want for coding, testing, and notebooks.

This is for *you* (as a developer) — not for production.

---

## 🔑 Why not just one image?

* **Best practice** is separation of concerns:

  * Production stays lean, fast, and stable.
  * Development stays flexible, with all the tools you want.
* If you only had one image, it would either be too heavy (if you included dev stuff), or too limited (if you left it out).

---

So the **two build steps** are just:

* “Make me the clean production-ready image” → `myproject:latest`
* “Make me the dev playground image” → `myproject:dev`




Nice 🚀 you’ve just built two images — let’s verify them step by step.

## Confirm Container Creation
---

### 🔍 Step 1: List all images

Run:

```bash
docker images
```

You should see both of your builds listed, something like:

```
REPOSITORY    TAG       IMAGE ID       CREATED          SIZE
myproject     latest    abcd1234efgh   2 minutes ago    300MB
myproject     dev       xyz9876ijkl    1 minute ago     600MB
```

* `myproject:latest` → your runtime image (leaner).
* `myproject:dev` → your dev image (larger, has Jupyter, pytest, etc.).

---

## 🔍 Step 2: Run the runtime image

Test if `main.py` works with your `.env`:

```bash
docker run --rm --env-file .env myproject:latest
```

Expected:

* It prints system info (Python version, OS, args).
* It sends *“Hello Claude, can you confirm you’re working?”* and Claude replies. 🎉

---

## 🔍 Step 3: Run the dev image (optional test)

Start Jupyter from inside the dev image:

```bash
docker run --rm -it -p 8888:8888 myproject:dev jupyter lab --ip=0.0.0.0 --allow-root
```

Expected:

* It will show you a JupyterLab URL with a token in the logs.
* Open that in your browser at [http://localhost:8888](http://localhost:8888).

---

✅ If both of those tests succeed, your build is confirmed good and your template is working.




# 💻 Basic Terminal Commands Cheat Sheet

### 📂 File & Folder Navigation

```bash
pwd               # print working directory
ls                # list files
ls -a             # list all files (including hidden)
cd foldername     # change directory
cd ..             # go up one level
```

### 📁 Create / Move / Delete

```bash
mkdir myfolder                # make a new folder
touch file.txt                # create empty file
mv oldname.txt newname.txt    # rename/move file
cp file.txt copy.txt          # copy file
rm file.txt                   # delete file
rm -r myfolder                # delete folder (recursive)
```

### 📄 Viewing Files

```bash
cat file.txt        # show entire file
less file.txt       # scroll through file (press q to quit)
head file.txt       # show first 10 lines
tail file.txt       # show last 10 lines
```

### 📝 Editing Files

```bash
nano file.txt       # edit file in nano (Ctrl+O save, Ctrl+X exit)
```

### ⚙️ System Info (inside container)

```bash
python --version    # check Python version
env                 # list environment variables
which python        # find where python is installed
```





## 🛑 Exiting a Running Container

It depends how you started the container:

### 1. If you ran it **interactively** (with `-it` and got a bash shell inside):

* You’re at a shell prompt like `root@<container-id>:/app#`.
* To exit cleanly:

  ```bash
  exit
  ```

  or press **Ctrl+D**.

That will stop the container unless you added `-d` (detached mode).

---

### 2. If you ran the container with a **default CMD** (non-interactive):

* It will exit automatically when the program finishes (e.g., when `python main.py` completes).
* You don’t need to do anything — check with:

  ```bash
  docker ps -a
  ```

---

### 3. If you ran in **detached mode** (`-d` flag):

* List running containers:

  ```bash
  docker ps
  ```
* Stop a container by ID:

  ```bash
  docker stop <container_id>
  ```

---

✅ In your case, since you’ve been running `docker run --rm ...` or `docker run -it ...`, you can just type:

```bash
exit
```

inside the container shell, and it will stop & auto-remove.






# 📂 Understanding `.` and `..` in the Filesystem

In Unix-like operating systems (Linux, macOS, inside Docker containers, etc.), every directory automatically contains two special entries:

### 1. `.` → **Current directory**

* Represents *“this directory itself”*.
* Useful when you want to explicitly refer to something in the folder you’re already in.
* Example:

  ```bash
  cd .
  ```

  Keeps you in the same directory (no visible effect).

  ```bash
  ls .
  ```

  Lists the contents of the current directory.

  ```bash
  ./script.py
  ```

  Runs `script.py` located in the current directory. (The `./` is needed because by default the shell doesn’t look in the current folder for executables.)

---

### 2. `..` → **Parent directory**

* Represents the *directory one level up*.

* Example:

  ```bash
  cd ..
  ```

  Moves you from the current folder into its parent.

* You can chain it:

  ```bash
  cd ../..
  ```

  Moves you up two levels in the directory tree.

---

### 🧠 Why they exist

* These entries are built into the filesystem.
* They’re why commands like `cd .` and `cd ..` work — the shell is literally treating them like directory names.
* Even in an empty folder, `.` and `..` always exist.

---

### 🔑 Quick Recap

* `.` → current directory
* `..` → parent directory
* They’re *not real files*, but special pointers provided by the filesystem.





## 🐳 In `docker build` commands

```bash
docker build -t myproject:latest .
```

* `docker build` → tells Docker to build an image.
* `-t myproject:latest` → gives the image a name and tag.
* The final `.` → means “use the current directory as the build context.”

---

## 🔑 What’s the *build context*?

* When you run `docker build`, Docker needs to know *what files* it can see and copy into the image.
* The directory you specify (`.` in this case) is called the **build context**.
* Docker sends everything in that directory (except what’s excluded in `.dockerignore`) into the build process.

So:

* `.` = “this directory”
* `..` = “the parent directory” (you could run `docker build ..` if the Dockerfile lived one level up)
* `/path/to/project` = absolute path if you want to build from somewhere else

---

## 🔎 Example

If your folder looks like this:

```
myproject/
├── Dockerfile
├── requirements.txt
├── main.py
└── data/
```

Running:

```bash
docker build -t myproject:latest .
```

* Docker sees the entire `myproject/` folder as the build context.
* Inside the `Dockerfile`, when you say:

  ```dockerfile
  COPY requirements.txt ./
  ```

  Docker knows to look for `requirements.txt` in that build context (the current folder).

---

✅ So yes — that trailing `.` is exactly the same `.` you’ve learned about: *the current directory*. In this case, it’s telling Docker “build from here.”

